# This notebook assumes that you've imported one or more NWB files into DataJoint 
# It allows you to run spikesorters on those data using the SpikeInterface package

#### Load all of the relevant modules

In [1]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2

import os
data_dir = '/Users/loren/data/nwb_builder_test_data'
os.environ['NWB_DATAJOINT_BASE_DIR'] = data_dir
os.environ['KACHERY_STORAGE_DIR'] = os.path.join(data_dir, 'kachery-storage')

import pynwb
import os

#DataJoint and DataJoint schema
import nwb_datajoint as nd
import datajoint as dj

import spikeinterface as si
import spikeextractors as se

env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
Connecting root@localhost:3306


16:52:45 [I] klustakwik KlustaKwik2 version 0.2.6


### Set the nwb file name and the name of the probe file to create from DataJoint

In [2]:
nwb_file_name = (nd.common.Session() & {'session_id': 'beans_01'}).fetch1('nwb_file_name')
probe_file_name = '/Users/loren/data/nwb_builder_test_data/test.prb'

In [3]:
nd.common.SortGroup().set_group_by_shank(nwb_file_name)


About to delete:
`common_ephys`.`sort_group__sort_group_electrode`: 256 items
`common_ephys`.`sort_group`: 8 items


Proceed? [yes, No]:  yes


Committed.


In [11]:
nd.common.SortGroup.SortGroupElectrode()

nwb_file_name the name of the NWB file,sort_group_id identifier for a group of electrodes,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode,"sort_reference_electrode the electrode to use for reference. -1: no reference, -2: common median"
beans20190718.nwb,0,0,0,-1
beans20190718.nwb,0,0,1,-1
beans20190718.nwb,0,0,2,-1
beans20190718.nwb,0,0,3,-1
beans20190718.nwb,0,0,4,-1
beans20190718.nwb,0,0,5,-1
beans20190718.nwb,0,0,6,-1
beans20190718.nwb,0,0,7,-1
beans20190718.nwb,0,0,8,-1
beans20190718.nwb,0,0,9,-1


In [3]:
nd.common.SortGroup().write_prb(nwb_file_name, probe_file_name)

In [4]:
recording = se.NwbRecordingExtractor(nd.common.Nwbfile.get_abs_path(nwb_file_name), electrical_series_name='e-series')



In [5]:
recording = recording.load_probe_file(probe_file_name)


#p = si.widgets.plot_electrode_geometry(recording)

In [6]:
nd.common.SpikeSorter().insert_from_spikeinterface()
nd.common.SpikeSorterParameters().insert_from_spikeinterface()



About to delete:
Nothing to delete


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/blob.py:180: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(obj, collections.ByteString):
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/blob.py:182: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(obj, collections.MutableSequence):
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/blob.py:184: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(obj, collections.Sequence):
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/blob.py:186: Deprecation

In [7]:
p = (nd.common.SpikeSorterParameters() & {'sorter_name': 'mountainsort4'}).fetch1()
print(p)
param = p['parameter_dict']
param['adjacency_radius'] = 100
param['curation'] = True
param['num_workers'] = 1

{'sorter_name': 'mountainsort4', 'parameter_set_name': 'default', 'parameter_dict': {'detect_sign': -1, 'adjacency_radius': -1, 'freq_min': 300, 'freq_max': 6000, 'filter': True, 'whiten': True, 'curation': False, 'num_workers': None, 'clip_size': 50, 'detect_threshold': 3, 'detect_interval': 10, 'noise_overlap_threshold': 0.15}}


In [8]:
first_shank_electrodes = (nd.common.SortGroup() & {'nwb_file_name' : nwb_file_name} & {'sort_group_id' : 0}).fetch('electrode_id')
first_shank_electrodes

AttributeError: module 'nwb_datajoint.common' has no attribute 'ElectrodeSortingInfo'

In [70]:
first_shank_recording = se.SubRecordingExtractor(recording, channel_ids=first_shank_electrodes)

In [ ]:
first_shank_sort = si.sorters.run_mountainsort4(recording=first_shank_recording, **param, grouping_property='group', output_folder='/Users/loren/data/nwb_builder_test_data/tmp')

'end_time' set to 138007758
'end_time' set to 138007758
